In [1]:
import os
import numpy as np
import pandas as pd
import re
import math
import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('train.csv')
updated_df = df.drop('price', axis=1)
print ('There are {} rows and {} columns in the dataset'.format(*updated_df.shape))
updated_df.head(3)


There are 33538 rows and 64 columns in the dataset


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,...,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,22267382,Modern and Cozy Large Studio in Brooklyn,Modern large studio with new amenities and app...,Our place is a little quiet sanctuary in the h...,Modern large studio with new amenities and app...,none,"BAM, Barclays, Brooklyn City Point, Fort Green...",NaN,"Subway: 2,3,4,5,A,C,B,Q,G",Washer/Dryer Dishwasher Internet Gym Roof Top ...,...,10.0,10.0,10.0,t,f,flexible,f,f,1,0.59
1,2473861,Royal Harlem TRIPLEX Home 5 Beds,Harlem is back and so gorgeous! Visit and expl...,Harlem is back and so gorgeous! Visit and expl...,Harlem is back and so gorgeous! Visit and expl...,none,HARLEM is a piece of real NY history overflowi...,HARLEM RESTAURANTS Red Rooster Harlem -- excel...,PUBLIC TRANSPORTATION: Conveniently near all p...,The WHOLE ENTIRE HOUSE,...,9.0,9.0,9.0,t,f,moderate,f,f,3,2.47
2,25079703,Sunny East Village Studio,"Clean, hip and well designed sun drenched East...",This is a rare East Village studio with it's h...,"Clean, hip and well designed sun drenched East...",none,East Village is one of the last remaining neig...,NaN,NaN,You'll have access to the entire space - it's ...,...,10.0,10.0,10.0,f,f,moderate,f,f,1,0.89


In [3]:
df.price

0         145.0
1         175.0
2         180.0
3          42.0
4          80.0
5         550.0
6         177.0
7          70.0
8         500.0
9         106.0
10        175.0
11         95.0
12         70.0
13         59.0
14        299.0
15        295.0
16         89.0
17        285.0
18         45.0
19        119.0
20        100.0
21         47.0
22         80.0
23         95.0
24        275.0
25         70.0
26        130.0
27         57.0
28         50.0
29         80.0
          ...  
33508      98.0
33509     100.0
33510     200.0
33511      60.0
33512      75.0
33513      35.0
33514     105.0
33515     109.0
33516      60.0
33517      75.0
33518     160.0
33519     200.0
33520      87.0
33521     125.0
33522     105.0
33523     995.0
33524      89.0
33525     135.0
33526      95.0
33527      42.0
33528      85.0
33529      65.0
33530     140.0
33531     265.0
33532      75.0
33533     110.0
33534    1195.0
33535      50.0
33536      60.0
33537     325.0
Name: price, Length: 335

In [4]:
df.columns

Index(['id', 'name', 'summary', 'space', 'description', 'experiences_offered',
       'neighborhood_overview', 'notes', 'transit', 'access', 'interaction',
       'house_rules', 'host_id', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_neighbourhood',
       'host_listings_count', 'host_verifications', 'host_has_profile_pic',
       'host_identity_verified', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'country_code', 'country', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet',
       'price', 'guests_included', 'extra_people', 'minimum_nights',
       'maximum_nights', 'number_of_reviews', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_ch

In [5]:
# drop all the unnecessary columns
feature_to_keep = ['id','description','access','property_type','room_type','accommodates','bathrooms',
                  'bedrooms','bed_type','guests_included','extra_people','minimum_nights',
                  'maximum_nights','cancellation_policy', 'neighbourhood_group_cleansed','host_is_superhost']

new_df = updated_df[feature_to_keep]

# remove the dollar sign before "cleaning_fee", "extra_people", "price" and change the datatype to numerical variables
feature_to_remove_dollar = ['extra_people']
new_df[feature_to_remove_dollar] = new_df[feature_to_remove_dollar].replace('\$','',regex = True)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [6]:
new_df.isna().sum()

id                                  0
description                       308
access                          14234
property_type                       0
room_type                           0
accommodates                        0
bathrooms                          60
bedrooms                           33
bed_type                            0
guests_included                     0
extra_people                        0
minimum_nights                      0
maximum_nights                      0
cancellation_policy                 0
neighbourhood_group_cleansed        0
host_is_superhost                   5
dtype: int64

In [7]:
# drop 308 rows with missing descriptions
new_df.loc[:,'description'] = new_df['description'].dropna()

# fill NaN with median value for 'bathrooms', 'bedrooms','price'
new_df.loc[:,'bathrooms'] = new_df.loc[:,'bathrooms'].fillna(new_df.loc[:,'bathrooms'].median())
new_df.loc[:,'bedrooms'] = new_df.loc[:,'bedrooms'].fillna(new_df.loc[:,'bedrooms'].median())
#new_df.loc[:, 'reviews_per_month'] = new_df.loc[:, 'reviews_per_month'].fillna(new_df.loc[:, 'reviews_per_month'].median())

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [8]:
def guests(x):
    if x == 1:
        return 'one'
    elif x == 2:
        return 'two'
    elif x == 3: 
        return 'three'
    elif x == 4:
        return 'four'
    else:
        return 'many'

In [9]:
saved = new_df['guests_included'].apply(guests)
new_df['guests_included'] = saved
new_df

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,description,access,property_type,room_type,accommodates,bathrooms,bedrooms,bed_type,guests_included,extra_people,minimum_nights,maximum_nights,cancellation_policy,neighbourhood_group_cleansed,host_is_superhost
0,22267382,Modern large studio with new amenities and app...,Washer/Dryer Dishwasher Internet Gym Roof Top ...,Loft,Entire home/apt,2,1.0,1.0,Real Bed,two,30.00,7,12,flexible,Brooklyn,f
1,2473861,Harlem is back and so gorgeous! Visit and expl...,The WHOLE ENTIRE HOUSE,House,Entire home/apt,8,1.0,3.0,Real Bed,three,19.00,3,30,moderate,Manhattan,f
2,25079703,"Clean, hip and well designed sun drenched East...",You'll have access to the entire space - it's ...,Apartment,Entire home/apt,2,1.0,0.0,Real Bed,one,0.00,2,1125,moderate,Manhattan,f
3,9342478,"Private, spacious, comfortable room in 2-bed f...",NaN,Apartment,Private room,1,1.0,1.0,Real Bed,one,0.00,3,1125,flexible,Brooklyn,f
4,4866426,"Comfy, quiet and big private room in a three b...",Your room has a very comfortable queen sized b...,Apartment,Private room,2,1.0,1.0,Real Bed,one,14.00,1,90,flexible,Brooklyn,t
5,19115443,"Spacious in size, suites feature a fully equip...",NaN,Apartment,Entire home/apt,6,2.5,2.0,Real Bed,one,0.00,30,1125,super_strict_60,Manhattan,f
6,12132427,"This super safe, super quiet, super convenient...",The apartment is located on the ground floor o...,Apartment,Entire home/apt,4,1.0,1.0,Real Bed,two,25.00,3,1125,moderate,Manhattan,f
7,27551517,Large Sunny room in a 1 BR apartment for 1 ONL...,ALL,Apartment,Private room,1,1.0,1.0,Real Bed,one,0.00,2,2,strict_14_with_grace_period,Manhattan,f
8,27016367,"**We are a event space, The price is for a 5 h...",NaN,Guesthouse,Entire home/apt,16,2.0,0.0,Real Bed,one,0.00,1,1,strict_14_with_grace_period,Queens,f
9,14028380,Beautiful and spacious newly renovated 1 bedro...,There's a separate entryway upstairs to the pr...,Apartment,Entire home/apt,4,1.0,1.0,Real Bed,two,25.00,1,1125,moderate,Brooklyn,f


In [10]:
new_df['property_type'].value_counts()

Apartment                 27060
House                      2471
Townhouse                  1086
Loft                       1035
Condominium                 822
Serviced apartment          437
Guest suite                 163
Other                        92
Bed and breakfast            68
Boutique hotel               68
Resort                       65
Guesthouse                   37
Hostel                       32
Hotel                        29
Villa                        19
Bungalow                     19
Tiny house                    9
Boat                          5
Aparthotel                    5
Tent                          3
Cottage                       3
Camper/RV                     3
Timeshare                     1
Island                        1
Cabin                         1
Houseboat                     1
Casa particular (Cuba)        1
Train                         1
Chalet                        1
Name: property_type, dtype: int64

In [11]:
# merge small catergories in property_type into one category "Other"
Other = ['Serviced apartment','Guest suite','Other','Boutique hotel','Bed and breakfast','Resort','Hotel','Guesthouse',
        'Hostel','Bungalow','Villa','Tiny house','Aparthotel','Boat', 'Tent', 'Cottage','Camper/RV','Casa particular (Cuba)',        
'Island',                        
'Timeshare',                     
'Chalet',                        
'Cabin',                         
'Houseboat',                     
'Train', 'Cave', 'Nature lodge', 'Earth house', 'Castle']
new_df['property_type'].loc[new_df['property_type'].isin(Other)] = "Other"

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [12]:
new_df['property_type'].value_counts()

Apartment      27060
House           2471
Townhouse       1086
Other           1064
Loft            1035
Condominium      822
Name: property_type, dtype: int64

In [13]:
new_df['room_type'].value_counts()

Entire home/apt    17536
Private room       15235
Shared room          767
Name: room_type, dtype: int64

In [14]:
new_df['bed_type'].value_counts()

Real Bed         32907
Futon              251
Pull-out Sofa      193
Airbed             130
Couch               57
Name: bed_type, dtype: int64

In [15]:
# merge small catergories in bed_type into one category "No Bed"
Other = ['Futon','Pull-out Sofa','Airbed','Couch']
new_df['bed_type'].loc[new_df['bed_type'].isin(Other)] = "No Bed"

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
new_df['bed_type'].value_counts()

Real Bed    32907
No Bed        631
Name: bed_type, dtype: int64

In [17]:
new_df['cancellation_policy'].value_counts()

strict_14_with_grace_period    15330
flexible                       10345
moderate                        7746
super_strict_60                   81
super_strict_30                   34
strict                             1
long_term                          1
Name: cancellation_policy, dtype: int64

In [18]:
# merge small catergories in cancellation_policy into one category "Other"
Other = ['super_strict_60','super_strict_30','strict','long_term']
new_df['cancellation_policy'].loc[new_df['cancellation_policy'].isin(Other)] = "Other"

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
new_df

,id,description,access,property_type,room_type,accommodates,bathrooms,bedrooms,bed_type,guests_included,extra_people,minimum_nights,maximum_nights,cancellation_policy,neighbourhood_group_cleansed,host_is_superhost
0,22267382,Modern large studio with new amenities and app...,Washer/Dryer Dishwasher Internet Gym Roof Top ...,Loft,Entire home/apt,2,1.0,1.0,Real Bed,two,30.00,7,12,flexible,Brooklyn,f
1,2473861,Harlem is back and so gorgeous! Visit and expl...,The WHOLE ENTIRE HOUSE,House,Entire home/apt,8,1.0,3.0,Real Bed,three,19.00,3,30,moderate,Manhattan,f
2,25079703,"Clean, hip and well designed sun drenched East...",You'll have access to the entire space - it's ...,Apartment,Entire home/apt,2,1.0,0.0,Real Bed,one,0.00,2,1125,moderate,Manhattan,f
3,9342478,"Private, spacious, comfortable room in 2-bed f...",NaN,Apartment,Private room,1,1.0,1.0,Real Bed,one,0.00,3,1125,flexible,Brooklyn,f
4,4866426,"Comfy, quiet and big private room in a three b...",Your room has a very comfortable queen sized b...,Apartment,Private room,2,1.0,1.0,Real Bed,one,14.00,1,90,flexible,Brooklyn,t
5,19115443,"Spacious in size, suites feature a fully equip...",NaN,Apartment,Entire home/apt,6,2.5,2.0,Real Bed,one,0.00,30,1125,Other,Manhattan,f
6,12132427,"This super safe, super quiet, super convenient...",The apartment is located on the ground floor o...,Apartment,Entire home/apt,4,1.0,1.0,Real Bed,two,25.00,3,1125,moderate,Manhattan,f
7,27551517,Large Sunny room in a 1 BR apartment for 1 ONL...,ALL,Apartment,Private room,1,1.0,1.0,Real Bed,one,0.00,2,2,strict_14_with_grace_period,Manhattan,f
8,27016367,"**We are a event space, The price is for a 5 h...",NaN,Other,Entire home/apt,16,2.0,0.0,Real Bed,one,0.00,1,1,strict_14_with_grace_period,Queens,f
9,14028380,Beautiful and spacious newly renovated 1 bedro...,There's a separate entryway upstairs to the pr...,Apartment,Entire home/apt,4,1.0,1.0,Real Bed,two,25.00,1,1125,moderate,Brooklyn,f


In [20]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

import json
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim.models.ldamodel import LdaModel

/opt/conda/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [21]:
def preprocess_text(corpus):
    processed_corpus = []
    english_words = set(nltk.corpus.words.words())
    english_stopwords = set(stopwords.words('english'))
    wordnet_lemmatizer = WordNetLemmatizer()
    tokenizer = RegexpTokenizer(r'[A-Za-z|!]+')
    for row in corpus:
        sentences = []
        word_tokens = tokenizer.tokenize(row)
        word_tokens_lower = [t.lower() for t in word_tokens]
        word_tokens_lower_english = [t for t in word_tokens_lower if t in english_words or not t.isalpha()]
        word_tokens_no_stops = [t for t in word_tokens_lower_english if not t in english_stopwords]
        word_tokens_no_stops_lemmatized = [wordnet_lemmatizer.lemmatize(t) for t in word_tokens_no_stops]
        for word in word_tokens_no_stops_lemmatized:
            if len(word) >= 2:
                sentences.append(word)
        processed_corpus.append(sentences)
    return processed_corpus

def pipline(processed_corpus):
    dictionary = Dictionary(processed_corpus)
    doc_term_matrix = [dictionary.doc2bow(listing) for listing in processed_corpus]
    return dictionary, doc_term_matrix

def lda_topic_model(doc_term_matrix,dictionary,num_topics = 3, passes = 2):
    LDA = LdaModel
    ldamodel = LDA(doc_term_matrix,num_topics = num_topics, id2word = dictionary, passes = passes)
    return ldamodel

def topic_feature(ldamodel, doc_term_matrix, df, new_col, num_topics):
    docTopicProbMat = ldamodel[doc_term_matrix]
    docTopicProbDf = pd.DataFrame(index = df.index, columns = range(0,num_topics))
    for i,doc in enumerate(docTopicProbMat):
        for topic in doc:
            docTopicProbDf.iloc[i,topic[0]] = topic[1]
    docTopicProbDf = docTopicProbDf.fillna(0)
    docTopicProbDf[new_col] = docTopicProbDf.idxmax(axis=1)
    df_topics = docTopicProbDf[new_col]
    df_new = pd.concat([df,df_topics],axis = 1)
    return df_new

In [22]:
import nltk
nltk.download('words')

[nltk_data] Downloading package words to /home/y1do/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [23]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/y1do/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [24]:
corpus_description = new_df['description'].astype(str)

# use nlp package to process the text in description
processed_corpus_description = preprocess_text(corpus_description)

# generate the doc_term_matrix for lda model
#dictionary_description, doc_term_matrix_description = pipline(processed_corpus_description)

# lda model for topic modeling
#ldamodel_description = lda_topic_model(doc_term_matrix_description,dictionary_description)

# add the topic feature to the dataframe
#final_df = topic_feature(ldamodel_description,doc_term_matrix_description,new_df,new_col = 'description_topic', num_topics =3)


In [25]:
# generate the doc_term_matrix for lda model
dictionary_description, doc_term_matrix_description = pipline(processed_corpus_description)

In [26]:
# lda model for topic modeling
ldamodel_description = lda_topic_model(doc_term_matrix_description,dictionary_description)

In [27]:
# add the topic feature to the dataframe
final_df = topic_feature(ldamodel_description,doc_term_matrix_description, new_df,new_col = 'description_topic', num_topics =4)

In [28]:
review_median = df['reviews_per_month'].median()
final_df['reviews_per_month'] = df['reviews_per_month'].fillna(review_median)
final_df['access'] = df['access']

In [29]:
def match(x):
    matches = ["full", "whole", "entire", "WHOLE", "ENTIRE"]
    if any(a in x for a in matches):
        return True
    else:
        return False

In [30]:
saved = df['access'].fillna('None')
df['access'] = saved
str_saved = df['access'].apply(match)

In [31]:
final_df['new_access'] = str_saved
final_df.head()

,id,description,access,property_type,room_type,accommodates,bathrooms,bedrooms,bed_type,guests_included,extra_people,minimum_nights,maximum_nights,cancellation_policy,neighbourhood_group_cleansed,host_is_superhost,description_topic,reviews_per_month,new_access
0,22267382,Modern large studio with new amenities and app...,Washer/Dryer Dishwasher Internet Gym Roof Top ...,Loft,Entire home/apt,2,1.0,1.0,Real Bed,two,30.00,7,12,flexible,Brooklyn,f,2,0.59,False
1,2473861,Harlem is back and so gorgeous! Visit and expl...,The WHOLE ENTIRE HOUSE,House,Entire home/apt,8,1.0,3.0,Real Bed,three,19.00,3,30,moderate,Manhattan,f,2,2.47,True
2,25079703,"Clean, hip and well designed sun drenched East...",You'll have access to the entire space - it's ...,Apartment,Entire home/apt,2,1.0,0.0,Real Bed,one,0.00,2,1125,moderate,Manhattan,f,2,0.89,True
3,9342478,"Private, spacious, comfortable room in 2-bed f...",NaN,Apartment,Private room,1,1.0,1.0,Real Bed,one,0.00,3,1125,flexible,Brooklyn,f,0,0.77,False
4,4866426,"Comfy, quiet and big private room in a three b...",Your room has a very comfortable queen sized b...,Apartment,Private room,2,1.0,1.0,Real Bed,one,14.00,1,90,flexible,Brooklyn,t,2,3.14,False


In [32]:
def bath_match(x):
    if x < 3:
        return 'not many'
    else:
        return 'many'

In [33]:
def bed_match(x):
    if x < 5:
        return 'not many'
    else:
        return 'many'

In [34]:
def acc_match(x):
    if x < 8:
        return 'not many'
    else:
        return 'many'

In [35]:
bath_saved = df['bathrooms'].apply(bath_match)
final_df['bath'] = bath_saved

bed_saved = df['bedrooms'].apply(bed_match)
final_df['bed'] = bed_saved

acc_saved = df['accommodates'].apply(acc_match)
final_df['acc'] = acc_saved

In [36]:
s_corpus_description = df['space'].astype(str)

# use nlp package to process the text in description
s_processed_corpus_description = preprocess_text(s_corpus_description)

In [37]:
# generate the doc_term_matrix for lda model
s_dictionary_description, s_doc_term_matrix_description = pipline(s_processed_corpus_description)

In [38]:
# lda model for topic modeling
s_ldamodel_description = lda_topic_model(s_doc_term_matrix_description, s_dictionary_description)

In [39]:
# add the topic feature to the dataframe
the_final_df = topic_feature(s_ldamodel_description,s_doc_term_matrix_description, final_df,new_col = 'space_topic', num_topics =3)

In [40]:
n_corpus_description = df['name'].astype(str)

# use nlp package to process the text in description
n_processed_corpus_description = preprocess_text(n_corpus_description)

In [41]:
# generate the doc_term_matrix for lda model
n_dictionary_description, n_doc_term_matrix_description = pipline(n_processed_corpus_description)

In [42]:
# lda model for topic modeling
n_ldamodel_description = lda_topic_model(n_doc_term_matrix_description, n_dictionary_description)

In [43]:
# add the topic feature to the dataframe
name_final_df = topic_feature(n_ldamodel_description,n_doc_term_matrix_description, the_final_df,new_col = 'name_topic', num_topics =3)

In [44]:
sum_corpus_description = df['summary'].astype(str)

# use nlp package to process the text in description
sum_processed_corpus_description = preprocess_text(sum_corpus_description)

In [45]:
# generate the doc_term_matrix for lda model
sum_dictionary_description, sum_doc_term_matrix_description = pipline(sum_processed_corpus_description)

In [46]:
# lda model for topic modeling
sum_ldamodel_description = lda_topic_model(sum_doc_term_matrix_description, sum_dictionary_description)

In [47]:
# add the topic feature to the dataframe
summary_final_df = topic_feature(sum_ldamodel_description,sum_doc_term_matrix_description, name_final_df,new_col = 'summary_topic', num_topics =3)

In [48]:
def accommodates(x):
    if x >= 7:
        return True
    else:
        return False

In [49]:
accom_saved = df['accommodates'].apply(accommodates)
summary_final_df['acc_lot?'] = accom_saved
summary_final_df.head(3)

,id,description,access,property_type,room_type,accommodates,bathrooms,bedrooms,bed_type,guests_included,...,description_topic,reviews_per_month,new_access,bath,bed,acc,space_topic,name_topic,summary_topic,acc_lot?
0,22267382,Modern large studio with new amenities and app...,Washer/Dryer Dishwasher Internet Gym Roof Top ...,Loft,Entire home/apt,2,1.0,1.0,Real Bed,two,...,2,0.59,False,not many,not many,not many,2,2,2,False
1,2473861,Harlem is back and so gorgeous! Visit and expl...,The WHOLE ENTIRE HOUSE,House,Entire home/apt,8,1.0,3.0,Real Bed,three,...,2,2.47,True,not many,not many,many,1,2,0,True
2,25079703,"Clean, hip and well designed sun drenched East...",You'll have access to the entire space - it's ...,Apartment,Entire home/apt,2,1.0,0.0,Real Bed,one,...,2,0.89,True,not many,not many,not many,1,2,0,False


In [503]:
data = df.groupby(['host_neighbourhood']).mean()['price'].sort_values()

really_cheap = data.loc[(data.values >= 0) & (data.values < 50)].index.tolist()
pretty_cheap = data.loc[(data.values >= 50) & (data.values < 75)].index.tolist()
cheap = data.loc[(data.values >= 75) & (data.values < 100)].index.tolist()
under_110 = data.loc[(data.values >= 100) & (data.values < 110)].index.tolist()
under_120 = data.loc[(data.values >= 110) & (data.values < 120)].index.tolist()
under_130 = data.loc[(data.values >= 120) & (data.values < 130)].index.tolist()
under_140 = data.loc[(data.values >= 130) & (data.values < 140)].index.tolist()
under_150 = data.loc[(data.values >= 140) & (data.values < 150)].index.tolist()
under_160 = data.loc[(data.values >= 150) & (data.values < 160)].index.tolist()
under_180 = data.loc[(data.values >= 160) & (data.values < 180)].index.tolist()
under_190 = data.loc[(data.values >= 180) & (data.values < 190)].index.tolist()
under_200 = data.loc[(data.values >= 190) & (data.values < 200)].index.tolist()
under_210 = data.loc[(data.values >= 200) & (data.values < 210)].index.tolist()
under_220 = data.loc[(data.values >= 210) & (data.values < 220)].index.tolist()
under_230 = data.loc[(data.values >= 220) & (data.values < 230)].index.tolist()
under_240 = data.loc[(data.values >= 230) & (data.values < 240)].index.tolist()
under_250 = data.loc[(data.values >= 240) & (data.values < 250)].index.tolist()
under_260 = data.loc[(data.values >= 250) & (data.values < 260)].index.tolist()
under_270 = data.loc[(data.values >= 260) & (data.values < 270)].index.tolist()
under_280 = data.loc[(data.values >= 270) & (data.values < 280)].index.tolist()
expensive_300 = data.loc[(data.values >= 280) & (data.values < 400)].index.tolist()
expensive_400 = data.loc[(data.values >= 400) & (data.values < 500)].index.tolist()
expensive_500 = data.loc[(data.values >= 500) & (data.values < 600)].index.tolist()
expensive_600 = data.loc[(data.values >= 600) & (data.values < 700)].index.tolist()
really_expensive = data.loc[data.values >= 700].index.tolist()

In [465]:
def host_cat(x):
    if x in really_cheap:
        return 'really cheap'
    elif x in pretty_cheap:
        return 'pretty cheap'
    elif x in cheap:
        return 'cheap'
    elif x in under_110:
        return 'under 110'
    elif x in under_120:
        return 'under 120'
    elif x in under_130:
        return 'under 130'
    elif x in under_140:
        return 'under 140'
    elif x in under_150:
        return 'under 150'
    elif x in under_160:
        return 'under 160'
    elif x in under_180:
        return 'under 180'
    elif x in under_190:
        return 'under 190'
    elif x in under_200:
        return 'under 200'
    elif x in under_110:
        return 'under 210'
    elif x in under_220:
        return 'under 220'
    elif x in under_230:
        return 'under 230'
    elif x in under_240:
        return 'under 240'
    elif x in under_250:
        return 'under 250'
    elif x in under_260:
        return 'under 260'
    elif x in under_270:
        return 'under 270'
    elif x in under_280:
        return 'under 280'
    elif x in expensive_300:
        return 'expensive_300'
    elif x in expensive_400:
        return 'expensive_400'
    elif x in expensive_500:
        return 'expensive_500'
    elif x in expensive_600:
        return 'expensive_600'
    else:
        return 'really expensive'

In [466]:
host_saved = df['host_neighbourhood'].apply(host_cat)
summary_final_df['host_cat'] = host_saved
#summary_final_df.head()

In [467]:
data = df.groupby(['neighbourhood_cleansed']).mean().price.sort_values()

In [468]:
under_50 = data.loc[(data.values >= 0) & (data.values < 50)].index.tolist()
under_100 = data.loc[(n_data.values >= 50) & (data.values < 100)].index.tolist()
under_125 = data.loc[(data.values >= 100) & (data.values < 125)].index.tolist()
under_150 = data.loc[(data.values >= 125) & (data.values < 150)].index.tolist()
under_175 = data.loc[(data.values >= 150) & (data.values < 175)].index.tolist()
under_200 = data.loc[(data.values >= 175) & (data.values < 200)].index.tolist()
under_225 = data.loc[(data.values >= 200) & (data.values < 225)].index.tolist()
under_250 = data.loc[(data.values >= 225) & (data.values < 250)].index.tolist()
under_275 = data.loc[(data.values >= 250) & (data.values < 275)].index.tolist()
under_300 = data.loc[(data.values >= 275) & (data.values < 300)].index.tolist()
under_400 = data.loc[(data.values >= 300) & (data.values < 400)].index.tolist()
over_400 = data.loc[(data.values >= 400) & (data.values < 800)].index.tolist()

In [469]:
def n_cat(x):
    if x in under_50:
        return 'under 50'
    elif x in under_100:
        return 'under 100'
    elif x in under_125:
        return 'under 125'
    elif x in under_150:
        return 'under 150'
    elif x in under_175:
        return 'under 175'
    elif x in under_200:
        return 'under 200'
    elif x in under_225:
        return 'under 225'
    elif x in under_250:
        return 'under 250'
    elif x in under_275:
        return 'under 275'
    elif x in under_300:
        return 'under 300'
    elif x in under_400:
        return 'under 400'
    else:
        return 'over 400'

In [470]:
n_saved = df['neighbourhood_cleansed'].apply(n_cat)
summary_final_df['n_cat'] = n_saved

In [157]:
df.groupby(['guests_included']).mean().price

guests_included
1     130.441134
2     145.416110
3     196.831155
4     241.356634
5     278.460432
6     333.238636
7     286.906250
8     421.814433
9     538.000000
10    454.440000
11    624.000000
12    706.750000
14    413.500000
15    300.000000
16    589.222222
Name: price, dtype: float64

In [158]:
def guest_classify(x):
    if (x >= 1) and (x < 4):
        return 'pretty cheap'
    elif (x >= 4) and (x < 8):
        return 'medium'
    else:
        return 'expensive'

In [165]:
guest_saved = df['guests_included'].apply(guest_classify)
summary_final_df['guest_cat'] = guest_saved
guest_saved.value_counts()

pretty cheap    31169
medium           2207
expensive         162
Name: guests_included, dtype: int64

In [172]:
df.groupby(['number_of_reviews']).mean().price.head(5)

number_of_reviews
0    169.699608
1    139.106537
2    130.456785
3    138.150798
4    134.669811
Name: price, dtype: float64

In [173]:
def reviews_num(x):
    if x < 170:
        return 'three_digit'
    else:
        return 'two_digit'

In [174]:
review_saved = df['number_of_reviews'].apply(reviews_num)
summary_final_df['review_cat'] = review_saved

In [200]:
df.groupby(['reviews_per_month']).mean().price.head(670)

reviews_per_month
0.01    179.333333
0.02    138.504132
0.03    123.884932
0.04    135.406439
0.05    126.687500
0.06    127.200521
0.07    123.778495
0.08    145.285714
0.09    155.651099
0.10    137.375810
0.11    143.649351
0.12    142.863636
0.13    142.531987
0.14    139.676375
0.15    139.442231
0.16    128.295833
0.17    131.920705
0.18    128.134615
0.19    129.487365
0.20    132.008333
0.21    136.173709
0.22    136.945946
0.23    138.040359
0.24    143.200000
0.25    145.204545
0.26    137.872928
0.27    133.691011
0.28    138.814815
0.29    129.412621
0.30    146.840237
           ...    
6.46     66.000000
6.47    108.333333
6.49     58.500000
6.50     75.000000
6.51     74.333333
6.52    125.200000
6.53    149.666667
6.55    116.400000
6.56    163.000000
6.57     70.000000
6.58     54.000000
6.59     35.000000
6.60     89.333333
6.61     74.000000
6.62     90.000000
6.63    133.000000
6.64     58.000000
6.65     75.000000
6.67    146.000000
6.68     90.000000
6.69     58.0

In [201]:
def rpm_cat(x):
    if (x >= 0.00) & (x < 4.97):
        return 'high'
    elif (x >= 4.97) & (x < 6.5):
        return 'medium'
    else:
        return 'low'

In [203]:
rpm_saved = summary_final_df['reviews_per_month'].apply(reviews_num)
summary_final_df['rpm_cat'] = rpm_saved

In [214]:
combined = df['accommodates'] + df['bedrooms'] + df['bathrooms']
com_df = pd.DataFrame(combined)
com_df['price'] = df['price']
com_df.groupby([0]).mean()

,price
0,
2.0,114.266917
2.5,74.793103
3.0,99.790140
3.5,77.513158
4.0,108.450643
4.5,99.803478
5.0,140.459173
5.5,133.266667
6.0,161.900199


In [505]:
def combined_cat(x):
    if x <= 3:
        return 'one'
    elif (x > 3) & (x <= 5):
        return 'two'
    elif (x > 5) & (x <= 7):
        return 'three'
    elif (x > 7) & (x <= 10):
        return 'four'
    elif (x > 10) & (x <= 12):
        return 'four'
    elif (x > 12) & (x <= 15):
        return 'four'
    else:
        return 'else'

In [506]:
combined_saved = com_df[0].apply(combined_cat)
summary_final_df['combined_cat'] = combined_saved

In [513]:
cols_to_keep = [
'accommodates','bathrooms','bedrooms','maximum_nights','minimum_nights','property_type', 'reviews_per_month',
                'bed_type','room_type','extra_people','new_access','cancellation_policy', 'acc_lot?', 'host_cat', 'n_cat',
                'neighbourhood_group_cleansed', 'description_topic', 'space_topic','name_topic', 'summary_topic', 
    'host_is_superhost', 'guest_cat', 'review_cat']
model_df = summary_final_df[cols_to_keep]

# convert strings to dummies
categorical_feats = ['property_type','room_type','bed_type', 'new_access','cancellation_policy',
                     'neighbourhood_group_cleansed',
                     'description_topic', 'space_topic','name_topic', 'summary_topic', 'acc_lot?', 
                     'host_cat', 'n_cat', 'host_is_superhost', 'guest_cat', 'review_cat']
model_df = pd.get_dummies(model_df,columns = categorical_feats,drop_first = False)

# separate the target variable "yield" from the dataset
target = df['price']
X_df = model_df

In [96]:
test_df = pd.read_csv('test.csv')
print ('There are {} rows and {} columns in the dataset'.format(*test_df.shape))
test_df.head(3)

There are 17337 rows and 64 columns in the dataset


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,...,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,19307997,Super Lux 2BR in Downtown Manhattan,Prepare to be WOWED! This spectacularly bright...,"Top of the line Wolf and Sub-Zero appliances, ...",Prepare to be WOWED! This spectacularly bright...,none,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,f,f,flexible,f,f,1,NaN
1,20176193,Vintage Eclectic Brownstone Pad in Brooklyn,"Ideal for romantic, creative types, this is an...","Not your typical New York abode, my apartment ...","Ideal for romantic, creative types, this is an...",none,Bed Stuy is a diverse historic neighborhood wi...,This is an actual unique living experience whe...,Close to buses and subways there is also free ...,"Entrance hallway, living room, bedroom, kitche...",...,10.0,10.0,10.0,f,f,flexible,f,f,1,1.48
2,19485371,Spacious Harlem Hideaway,"Postive Vibes . This is our Harlem tree house,...",The private room is very spacious and cozy. Th...,"Postive Vibes . This is our Harlem tree house,...",none,You are in a Cultural Haven full of restaurant...,We also keep cucumber water in the fridge feel...,"Train, uber or a taxi. (Extremely taxi accessi...","Private Room, Kitchen And Bathroom",...,10.0,10.0,10.0,t,f,flexible,f,f,1,0.37


In [97]:
# drop all the unnecessary columns
test_feature_to_keep = ['id','description','property_type','room_type','accommodates','bathrooms',
                  'bedrooms','bed_type','guests_included','extra_people','minimum_nights',
                  'maximum_nights','reviews_per_month', 'cancellation_policy','neighbourhood_group_cleansed','host_is_superhost']

test_new_df = test_df[test_feature_to_keep]

# remove the dollar sign before "cleaning_fee", "extra_people", "price" and change the datatype to numerical variables
test_feature_to_remove_dollar = ['extra_people']
test_new_df.loc[:,test_feature_to_remove_dollar] = test_new_df.loc[:,test_feature_to_remove_dollar].replace('\$','',regex = True)
test_new_df.loc[:,test_feature_to_remove_dollar] = test_new_df.loc[:,test_feature_to_remove_dollar].apply(pd.to_numeric,errors = "coerce")

<>:10: DeprecationWarning: invalid escape sequence \$
<>:10: DeprecationWarning: invalid escape sequence \$
<>:10: DeprecationWarning: invalid escape sequence \$
<ipython-input-97-69ab648beaf6>:10: DeprecationWarning: invalid escape sequence \$
  test_new_df.loc[:,test_feature_to_remove_dollar] = test_new_df.loc[:,test_feature_to_remove_dollar].replace('\$','',regex = True)
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [98]:
test_new_df.isna().sum()

id                                 0
description                      159
property_type                      0
room_type                          0
accommodates                       0
bathrooms                         36
bedrooms                          23
bed_type                           0
guests_included                    0
extra_people                       0
minimum_nights                     0
maximum_nights                     0
reviews_per_month               3632
cancellation_policy                0
neighbourhood_group_cleansed       0
host_is_superhost                  3
dtype: int64

In [99]:
# drop 159 rows with missing descriptions
test_new_df.loc[:,'description'] = test_new_df['description'].dropna()

# fill NaN with median value for 'bathrooms', 'bedrooms','price'
test_new_df.loc[:,'bathrooms'] = test_new_df.loc[:,'bathrooms'].fillna(test_new_df.loc[:,'bathrooms'].median())
test_new_df.loc[:,'bedrooms'] = test_new_df.loc[:,'bedrooms'].fillna(test_new_df.loc[:,'bedrooms'].median())
test_new_df.loc[:,'reviews_per_month'] = test_new_df.loc[:,'reviews_per_month'].fillna(test_new_df.loc[:,'reviews_per_month'].median())

In [100]:
test_new_df['property_type'].value_counts()

Apartment                 13932
House                      1281
Townhouse                   541
Loft                        509
Condominium                 470
Serviced apartment          230
Guest suite                  83
Other                        59
Bed and breakfast            45
Resort                       38
Boutique hotel               37
Hostel                       31
Guesthouse                   22
Hotel                        13
Bungalow                     11
Villa                        10
Tiny house                    6
Aparthotel                    4
Cabin                         3
Camper/RV                     2
Boat                          2
Cottage                       2
Castle                        1
Nature lodge                  1
Casa particular (Cuba)        1
Cave                          1
Tent                          1
Earth house                   1
Name: property_type, dtype: int64

In [101]:
# merge small catergories in property_type into one category "Other"
test_Other = ['Serviced apartment','Bed and breakfast','Resort','Hotel','Guesthouse','Other','Hostel', 'Guest suite', 'Boutique hotel', 
'Bungalow','Villa','Tiny house','Aparthotel','Boat', 'Tent', 'Cottage','Camper/RV','Casa particular (Cuba)', 'Nature lodge',       
'Island','Castle', 'Earth house','Cabin', 'Cave']

test_new_df['property_type'].loc[test_new_df['property_type'].isin(test_Other)] = "Other"

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [102]:
test_new_df['property_type'].value_counts()

Apartment      13932
House           1281
Other            604
Townhouse        541
Loft             509
Condominium      470
Name: property_type, dtype: int64

In [103]:
test_new_df['room_type'].value_counts()

Entire home/apt    8982
Private room       7964
Shared room         391
Name: room_type, dtype: int64

In [104]:
test_new_df['bed_type'].value_counts()

Real Bed         16982
Pull-out Sofa      118
Futon              113
Airbed              89
Couch               35
Name: bed_type, dtype: int64

In [105]:
# merge small catergories in bed_type into one category "No Bed"
Other = ['Futon','Pull-out Sofa','Airbed','Couch']
test_new_df['bed_type'].loc[test_new_df['bed_type'].isin(Other)] = "No Bed"

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [106]:
test_new_df['bed_type'].value_counts()

Real Bed    16982
No Bed        355
Name: bed_type, dtype: int64

In [107]:
test_new_df['cancellation_policy'].value_counts()

strict_14_with_grace_period    7934
flexible                       5406
moderate                       3936
super_strict_60                  44
super_strict_30                  16
strict                            1
Name: cancellation_policy, dtype: int64

In [108]:
# merge small catergories in cancellation_policy into one category "Other"
Other = ['super_strict_60','super_strict_30','strict','long_term']
test_new_df['cancellation_policy'].loc[test_new_df['cancellation_policy'].isin(Other)] = "Other"

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [109]:
test_new_df['cancellation_policy'].value_counts()

strict_14_with_grace_period    7934
flexible                       5406
moderate                       3936
Other                            61
Name: cancellation_policy, dtype: int64

In [110]:
test_corpus_description = test_new_df['description'].astype(str)

# use nlp package to process the text in description
test_processed_corpus_description = preprocess_text(test_corpus_description)

In [111]:
# generate the doc_term_matrix for lda model
test_dictionary_description, test_doc_term_matrix_description = pipline(test_processed_corpus_description)


In [112]:
# lda model for topic modeling
test_ldamodel_description = lda_topic_model(test_doc_term_matrix_description, test_dictionary_description)


In [113]:
# add the topic feature to the dataframe
test_final_df = topic_feature(test_ldamodel_description, test_doc_term_matrix_description, test_new_df, new_col = 'description_topic', num_topics =4)

In [114]:
review_median = test_df['reviews_per_month'].median()
test_final_df['reviews_per_month'] = test_df['reviews_per_month'].fillna(review_median)
test_final_df['access'] = test_df['access']

In [115]:
test_saved =test_df['access'].fillna('None')
test_df['access'] = saved
test_str_saved = test_df['access'].apply(match)

In [116]:
test_final_df['new_access'] = test_str_saved

In [117]:
test_bath_saved = test_df['bathrooms'].apply(bath_match)
test_final_df['bath'] = test_bath_saved

test_bed_saved = test_df['bedrooms'].apply(bed_match)
test_final_df['bed'] = test_bed_saved

test_acc_saved = test_df['accommodates'].apply(acc_match)
test_final_df['acc'] = test_acc_saved

In [118]:
test_s_corpus_description = test_df['space'].astype(str)

# use nlp package to process the text in description
test_s_processed_corpus_description = preprocess_text(test_s_corpus_description)

In [119]:
# generate the doc_term_matrix for lda model
test_s_dictionary_description, test_s_doc_term_matrix_description = pipline(test_s_processed_corpus_description)

In [120]:
# lda model for topic modeling
test_s_ldamodel_description = lda_topic_model(test_s_doc_term_matrix_description, test_s_dictionary_description)

In [121]:
# add the topic feature to the dataframe
test_the_final_df = topic_feature(test_s_ldamodel_description,test_s_doc_term_matrix_description, test_final_df,new_col = 'space_topic', num_topics =3)

In [128]:
test_n_corpus_description = test_df['name'].astype(str)
# use nlp package to process the text in description
test_n_processed_corpus_description = preprocess_text(test_n_corpus_description)

In [129]:
# generate the doc_term_matrix for lda model
test_n_dictionary_description, test_n_doc_term_matrix_description = pipline(test_n_processed_corpus_description)

In [130]:
# lda model for topic modeling
test_n_ldamodel_description = lda_topic_model(test_n_doc_term_matrix_description, test_n_dictionary_description)

In [131]:
# add the topic feature to the dataframe
test_name_final_df = topic_feature(test_n_ldamodel_description,test_n_doc_term_matrix_description, test_the_final_df,new_col = 'name_topic', num_topics =3)

In [132]:
test_sum_corpus_description = test_df['summary'].astype(str)
# use nlp package to process the text in description
test_sum_processed_corpus_description = preprocess_text(test_sum_corpus_description)

In [133]:
# generate the doc_term_matrix for lda model
test_sum_dictionary_description, test_sum_doc_term_matrix_description = pipline(test_sum_processed_corpus_description)

In [134]:
# lda model for topic modeling
test_sum_ldamodel_description = lda_topic_model(test_sum_doc_term_matrix_description, test_sum_dictionary_description)

In [135]:
# add the topic feature to the dataframe
test_summary_final_df = topic_feature(test_sum_ldamodel_description,test_sum_doc_term_matrix_description, test_name_final_df,new_col = 'summary_topic', num_topics =3)

In [136]:
test_accom_saved = test_df['accommodates'].apply(accommodates)
test_summary_final_df['acc_lot?'] = test_accom_saved
test_summary_final_df.head(3)

,id,description,property_type,room_type,accommodates,bathrooms,bedrooms,bed_type,guests_included,extra_people,...,description_topic,access,new_access,bath,bed,acc,space_topic,name_topic,summary_topic,acc_lot?
0,19307997,Prepare to be WOWED! This spectacularly bright...,Condominium,Entire home/apt,4,2.0,2.0,Real Bed,1,0.0,...,2,NaN,False,not many,not many,not many,2,2,1,False
1,20176193,"Ideal for romantic, creative types, this is an...",Apartment,Entire home/apt,2,1.0,2.0,Real Bed,1,0.0,...,0,"Entrance hallway, living room, bedroom, kitche...",True,not many,not many,not many,2,0,0,False
2,19485371,"Postive Vibes . This is our Harlem tree house,...",Apartment,Private room,1,1.0,1.0,Real Bed,1,20.0,...,0,"Private Room, Kitchen And Bathroom",True,not many,not many,not many,2,0,2,False


In [474]:
test_host_saved = test_df['host_neighbourhood'].apply(host_cat)
test_summary_final_df['host_cat'] = test_host_saved
#summary_final_df.head()

In [475]:
test_n_saved = test_df['neighbourhood_cleansed'].apply(n_cat)
test_summary_final_df['n_cat'] = test_n_saved

In [457]:
test_guest_saved = test_df['guests_included'].apply(guest_classify)
test_summary_final_df['guest_cat'] = test_guest_saved

In [458]:
test_review_saved = test_df['number_of_reviews'].apply(reviews_num)
test_summary_final_df['review_cat'] = test_review_saved

In [476]:
test_cols_to_keep = ['accommodates','bathrooms','bedrooms','extra_people','maximum_nights','minimum_nights','property_type',
                'bed_type','room_type','cancellation_policy', 'reviews_per_month', 'new_access', 'acc_lot?', 'host_cat', 'n_cat',
                     'neighbourhood_group_cleansed', 'description_topic', 'space_topic', 'name_topic', 'summary_topic', 
                     'host_is_superhost', 'guest_cat', 'review_cat']
test_model_df = test_summary_final_df[test_cols_to_keep]

# convert strings to dummies
test_categorical_feats = ['property_type','room_type','bed_type', 'new_access','cancellation_policy', 'acc_lot?',
                           'description_topic', 'space_topic', 'name_topic', 'summary_topic', 'host_cat', 'n_cat',
                          'host_is_superhost', 'neighbourhood_group_cleansed', 'guest_cat', 'review_cat']
test_model_df = pd.get_dummies(test_model_df,columns = test_categorical_feats,drop_first = False)

test_X_df = test_model_df

In [477]:
print(test_X_df.shape)
print(X_df.shape)

(17337, 85)
(33538, 85)


In [479]:
test_X_df = test_X_df[['accommodates', 'bathrooms', 'bedrooms', 'maximum_nights',
       'minimum_nights', 'reviews_per_month', 'extra_people',
       'property_type_Apartment', 'property_type_Condominium',
       'property_type_House', 'property_type_Loft', 'property_type_Other',
       'property_type_Townhouse', 'room_type_Entire home/apt',
       'room_type_Private room', 'room_type_Shared room', 'bed_type_No Bed',
       'bed_type_Real Bed', 'new_access_False', 'new_access_True',
       'cancellation_policy_Other', 'cancellation_policy_flexible',
       'cancellation_policy_moderate',
       'cancellation_policy_strict_14_with_grace_period',
       'neighbourhood_group_cleansed_Bronx',
       'neighbourhood_group_cleansed_Brooklyn',
       'neighbourhood_group_cleansed_Manhattan',
       'neighbourhood_group_cleansed_Queens',
       'neighbourhood_group_cleansed_Staten Island', 'description_topic_0',
       'description_topic_1', 'description_topic_2', 'space_topic_0',
       'space_topic_1', 'space_topic_2', 'name_topic_0', 'name_topic_1',
       'name_topic_2', 'summary_topic_0', 'summary_topic_1', 'summary_topic_2',
       'acc_lot?_False', 'acc_lot?_True', 'host_cat_cheap',
       'host_cat_expensive_300', 'host_cat_expensive_400',
       'host_cat_expensive_500', 'host_cat_pretty cheap',
       'host_cat_really cheap', 'host_cat_really expensive',
       'host_cat_under 110', 'host_cat_under 120', 'host_cat_under 130',
       'host_cat_under 140', 'host_cat_under 150', 'host_cat_under 160',
       'host_cat_under 180', 'host_cat_under 190', 'host_cat_under 200',
       'host_cat_under 220', 'host_cat_under 230', 'host_cat_under 240',
       'host_cat_under 250', 'host_cat_under 260', 'host_cat_under 270',
       'host_cat_under 280', 'n_cat_over 400', 'n_cat_under 100',
       'n_cat_under 125', 'n_cat_under 150', 'n_cat_under 175',
       'n_cat_under 200', 'n_cat_under 225', 'n_cat_under 250',
       'n_cat_under 275', 'n_cat_under 300', 'n_cat_under 400',
       'n_cat_under 50', 'host_is_superhost_f', 'host_is_superhost_t',
       'guest_cat_expensive', 'guest_cat_medium', 'guest_cat_pretty cheap',
       'review_cat_three_digit', 'review_cat_two_digit']]

In [508]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

#seed = 42
#X_train,X_test,y_train,y_test = train_test_split(X_df,target,random_state=seed)
linreg = LinearRegression(n_jobs=5).fit(X_df, target)
y_pred_linreg = linreg.predict(X_df)
print("Root Mean squared error: %.3f" %np.sqrt(mean_squared_error(target,y_pred_linreg)))
#rmse_lr = np.sqrt(mean_squared_error(target,y_pred_linreg))
#print(X_df.columns)
#print(test_X_df.columns)
y_pred_linreg

Root Mean squared error: 95.419


array([179.99845461, 257.48651766, 155.90748882, ...,  20.10803818,
        52.01927556, 364.71535913])

In [514]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(n_estimators=100).fit(X_df, target)
y_pred_rf = rf_reg.predict(X_df)
print("Root Mean squared error: %.3f" %np.sqrt(mean_squared_error(target,y_pred_rf)))
#rmse_rf = np.sqrt(mean_squared_error(y_test,y_pred_linreg))
y_pred_rf

Root Mean squared error: 33.199


array([149.35 , 187.36 , 165.5  , ...,  87.065,  59.47 , 367.85 ])

In [530]:
from sklearn.ensemble import GradientBoostingRegressor

de_reg = GradientBoostingRegressor(n_estimators=220).fit(X_df, target)
y_pred_dt = de_reg.predict(test_X_df)
#print("Root Mean squared error: %.3f" %np.sqrt(mean_squared_error(target,y_pred_dt)))
y_pred_dt

array([434.7409996 , 127.41651705,  68.85274452, ...,  78.2630413 ,
        44.86192879, 144.18402125])

In [502]:
from sklearn.tree import DecisionTreeRegressor

t_reg = DecisionTreeRegressor().fit(X_df, target)
y_pred_t = t_reg.predict(X_df)
print("Root Mean squared error: %.3f" %np.sqrt(mean_squared_error(target,y_pred_t)))
y_pred_t

Root Mean squared error: 6.273


array([145., 175., 180., ...,  50.,  60., 325.])

In [531]:
output_test = pd.DataFrame()
output_test['Id'] = test_df.id
output_test['Predicted'] = y_pred_dt
output_test.to_csv('test_pred_38.csv', index = False)